In [20]:
import pandas as pd

import emission.core.get_database as edb
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.storage.decorations.trip_queries as esdt
import emission.storage.decorations.timeline as esdl
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
import scaffolding
from uuid import UUID

# %conda install geopandas

import geopandas as gpd
from collections import defaultdict
import shapely as shp
import matplotlib.pyplot as plt

%matplotlib inline

import folium
import folium.plugins as fpl
import numpy as np
import branca.element as bre
import functools


In [21]:
year = None
month = None
# program = "study"
study_type = "study"
mode_of_interest = "e-bike"
include_test_users = False #NO TEST USERS

dynamic_labels = {
    "MODE": [
      {"value":"walk", "baseMode":"WALKING", "met_equivalent":"WALKING", "kgCo2PerKm": 0},
      {"value":"e-auto_rickshaw", "baseMode":"MOPED", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.085416859},
      {"value":"auto_rickshaw", "baseMode":"MOPED", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.231943784},
      {"value":"motorcycle", "baseMode":"MOPED", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.113143309},
      {"value":"e-bike", "baseMode":"E_BIKE", "met": {"ALL": {"range": [0, -1], "mets": 4.9}}, "kgCo2PerKm": 0.00728},
      {"value":"bike", "baseMode":"BICYCLING", "met_equivalent":"BICYCLING", "kgCo2PerKm": 0},
      {"value":"drove_alone", "baseMode":"CAR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.22031},
      {"value":"shared_ride", "baseMode":"CAR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.11015},
      {"value":"e_car_drove_alone", "baseMode":"E_CAR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.08216},
      {"value":"e_car_shared_ride", "baseMode":"E_CAR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.04108},
      {"value":"taxi", "baseMode":"TAXI", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.30741},
      {"value":"bus", "baseMode":"BUS", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.20727},
      {"value":"train", "baseMode":"TRAIN", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.12256},
      {"value":"free_shuttle", "baseMode":"BUS", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.20727},
      {"value":"air", "baseMode":"AIR", "met_equivalent":"IN_VEHICLE", "kgCo2PerKm": 0.09975},
      {"value":"not_a_trip", "baseMode":"UNKNOWN", "met_equivalent":"UNKNOWN", "kgCo2PerKm": 0},
      {"value":"other", "baseMode":"OTHER", "met_equivalent":"UNKNOWN", "kgCo2PerKm": 0}
    ],
    "PURPOSE": [
      {"value":"home"},
      {"value":"work"},
      {"value":"at_work"},
      {"value":"school"},
      {"value":"transit_transfer"},
      {"value":"shopping"},
      {"value":"meal"},
      {"value":"pick_drop_person"},
      {"value":"pick_drop_item"},
      {"value":"personal_med"},
      {"value":"access_recreation"},
      {"value":"exercise"},
      {"value":"entertainment"},
      {"value":"religious"},
      {"value":"other"}
    ],
    "translations": {
      "en": {
        "walk": "Walk",
        "e-auto_rickshaw":"E-tuk tuk",
        "auto_rickshaw":"Tuk Tuk",
        "motorcycle":"Motorcycle",
        "e-bike": "E-bike",
        "bike": "Bicycle",
        "drove_alone": "Car Drove Alone",
        "shared_ride": "Car Shared Ride",
        "e_car_drove_alone": "E-Car Drove Alone",
        "e_car_shared_ride": "E-Car Shared Ride",
        "taxi": "Taxi/Loca/inDrive",
        "bus": "Bus",
        "train": "Train",
        "free_shuttle": "Free Shuttle",
        "air": "Airplane",
        "not_a_trip": "Not a trip",
        "home": "Home",
        "work": "To Work",
        "at_work": "At Work",
        "school": "School",
        "transit_transfer": "Transit transfer",
        "shopping": "Shopping",
        "meal": "Meal",
        "pick_drop_person": "Pick-up/ Drop off Person",
        "pick_drop_item": "Pick-up/ Drop off Item",
        "personal_med": "Personal/ Medical",
        "access_recreation": "Access Recreation",
        "exercise": "Recreation/ Exercise",
        "entertainment": "Entertainment/ Social",
        "religious": "Religious",
        "other": "Other"
      },
      "lo": {
        "walk": "ດ້ວຍການຍ່າງ",
        "e-auto_rickshaw":"ລົດ 3 ລໍ້ໄຟຟ້າ ຫລື ຕຸກຕຸກໄຟຟ້າ",
        "auto_rickshaw":"ເດີນທາດ້ວຍ ລົດຕຸກຕຸກ ຫລື ລົດສາມລໍ້",
        "motorcycle":"ລົດຈັກ",
        "e-bike": "ວຍລົດຈັກໄຟຟ້າ",
        "bike": "ລົດຖີບ",
        "drove_alone": "ເດີນທາງ ດ້ວຍລົດໃຫ່ຍ ເຊີ່ງເປັນລົດທີ່ຂັບເອງ",
        "shared_ride": "ເດີນທາງດ້ວຍລົດໃຫ່ຍ ຮ່ວມກັບລົດຄົນອຶ່ນ",
        "e_car_drove_alone": "ດ້ວຍການຂັບລົດໄຟຟ້າໄປເອງ",
        "e_car_shared_ride": "ດ້ວຍການຈ້າງລົດໄຟຟ້າໄປ",
        "taxi": "ແທັກຊີ",
        "bus": "ລົດເມ",
        "train": "ລົດໄຟ",
        "free_shuttle": "ລົດຮັບສົ່ງຟຣີ",
        "air": "ຍົນ",
        "not_a_trip": "ບໍ່ແມ່ນການເດີນທາງ",
        "home": "ບ້ານ",
        "work": "ໄປເຮັດວຽກ",
        "at_work": "ຢູ່ບ່ອນເຮັດວຽກ",
        "school": "ໄປໂຮງຮຽນ",
        "transit_transfer": "ການຖ່າຍໂອນການເດີນທາງ",
        "shopping": "ຊອບປິ້ງ",
        "meal": "ອາຫານ",
        "pick_drop_person": "ໄປຮັບ ຫລື ສົນ ຄົນ",
        "pick_drop_item": "ໄປຮັບ ຫລື ສົ່ງສິນຄ້າ",
        "personal_med": "ໄປຫາໝໍ",
        "access_recreation": "ເຂົ້າເຖິງການພັກຜ່ອນ",
        "exercise": "ພັກຜ່ອນ/ອອກກຳລັງກາຍ",
        "entertainment": "ບັນເທີງ/ສັງຄົມ",
        "religious": "ຈຸດປະສົງທາງສາດສະໜາ",
        "other": "ອື່ນໆ"
      }
    }
  }

## Load the programs

In [22]:
# Split UUIDs by program
program_uuid_map = {}
for ue in edb.get_uuid_db().find():
    program = ue['user_email'].split("_")[0]
    if program in program_uuid_map.keys():
        program_uuid_map[program].append(str(ue['uuid']))
    else:
        print(f"Found new program {program}, creating new list")
        program_uuid_map[program] = []
        program_uuid_map[program].append(str(ue['uuid']))

uuid_program_list = []
for ue in edb.get_uuid_db().find():
    program = ue['user_email'].split("_")[0]
    uuid_program_list.append({"program": program, "opcode": ue["user_email"], "user_id_str": str(ue['uuid'])})

uuid_program_df = pd.DataFrame.from_dict(uuid_program_list)
uuid_program_df.head()

Found new program nrelop, creating new list


,program,opcode,user_id_str
0,nrelop,nrelop_usaid-laos-ev_testingUpdates,8cab6027-0e64-4e34-ab34-2d40b345a407
1,nrelop,nrelop_usaid-laos-ev_testUpdates,6a1a67ae-cd43-423d-860f-eb711ebb7327
2,nrelop,nrelop_usaid-laos-ev_test_testUpdates,7f0f0946-d0b1-4b12-8e9c-3cb9d7d87fad
3,nrelop,nrelop_usaid-laos-ev_test_oct9Demo,7ccc54e0-fe15-45f3-a8db-41f661ec3ba4
4,nrelop,nrelop_usaid-laos-ev_oct9Demo,b621c36f-3774-4d41-891c-b16e28907aa5


In [46]:
#workaround for custom labels :)
def mapping_labels(dynamic_labels, label_type):
    if "translations" in dynamic_labels and "en" in dynamic_labels["translations"]:
        translations = dynamic_labels["translations"]["en"]
        dic_mapping = dict()

        def translate_labels(labels):
            translation_mapping = {}
            for label in labels:
                value = label["value"]
                translation = translations.get(value)
                translation_mapping[value] = translation
            return defaultdict(lambda: 'Other', translation_mapping)
        dic_mapping = translate_labels(dynamic_labels[label_type])
        print(dic_mapping)
        return dic_mapping

dic_re = mapping_labels(dynamic_labels, 'MODE')

dic_pur = mapping_labels(dynamic_labels, 'PURPOSE')


defaultdict(<function mapping_labels.<locals>.translate_labels.<locals>.<lambda> at 0x7f326282a430>, {'walk': 'Walk', 'e-auto_rickshaw': 'E-tuk tuk', 'auto_rickshaw': 'Tuk Tuk', 'motorcycle': 'Motorcycle', 'e-bike': 'E-bike', 'bike': 'Bicycle', 'drove_alone': 'Car Drove Alone', 'shared_ride': 'Car Shared Ride', 'e_car_drove_alone': 'E-Car Drove Alone', 'e_car_shared_ride': 'E-Car Shared Ride', 'taxi': 'Taxi/Loca/inDrive', 'bus': 'Bus', 'train': 'Train', 'free_shuttle': 'Free Shuttle', 'air': 'Airplane', 'not_a_trip': 'Not a trip', 'other': 'Other'})
defaultdict(<function mapping_labels.<locals>.translate_labels.<locals>.<lambda> at 0x7f3279088a60>, {'home': 'Home', 'work': 'To Work', 'at_work': 'At Work', 'school': 'School', 'transit_transfer': 'Transit transfer', 'shopping': 'Shopping', 'meal': 'Meal', 'pick_drop_person': 'Pick-up/ Drop off Person', 'pick_drop_item': 'Pick-up/ Drop off Item', 'personal_med': 'Personal/ Medical', 'access_recreation': 'Access Recreation', 'exercise': 'R

## Load the trip table

In [47]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dynamic_labels=dynamic_labels,
                                                                            dic_re=dic_re,
                                                                            dic_pur=dic_pur,
                                                                            include_test_users=include_test_users)
tq = scaffolding.get_time_query(year, month)
all_trips = scaffolding.load_all_participant_trips(program, tq, False)

# (year, month, program, study_type, dynamic_labels, dic_re, dic_pur=None, include_test_users=False)

0                   nrelop_usaid-laos-ev_testingUpdates
1                      nrelop_usaid-laos-ev_testUpdates
4                         nrelop_usaid-laos-ev_oct9Demo
5     nrelop_usaid-laos-ev_default_NZPQovdgPI2OywmFF...
6                   nrelop_usaid-laos-ev_default_test-1
7     nrelop_usaid-laos-ev_default_jMbTtd5Qbjuw9e5BR...
8     nrelop_usaid-laos-ev_default_3JaVzbPwvJjXE8x1o...
9     nrelop_usaid-laos-ev_default_G6CznZqlNHaxJHiev...
10    nrelop_usaid-laos-ev_default_s5GK8IC6ljKwk6xZb...
11    nrelop_usaid-laos-ev_default_XTFCs7rHV0dxv2UKD...
12    nrelop_usaid-laos-ev_default_5ubmjSfP4hG09QubF...
13    nrelop_usaid-laos-ev_default_4lOYSVNYr56vXy0zS...
14              nrelop_usaid-laos-ev_testingOnTestPhone
15                nrelop_usaid-laos-ev_testingTheConfig
16    nrelop_usaid-laos-ev_default_QDnNcBapYzOCfMvCF...
17    nrelop_usaid-laos-ev_default_uVKy82hyFA88KJwUk...
18                       nrelop_usaid-laos-ev_testPhone
19    nrelop_usaid-laos-ev_default_Ioa3JqXWR42qY

Loaded all confirmed trips of length 9243


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,7,27,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,7,27,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,7,27,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,7,26,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,7,26,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09


After filtering, found 9243 participant trips 


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,7,27,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,7,27,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,7,27,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,7,26,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,7,26,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09


...

0                   nrelop_usaid-laos-ev_testingUpdates
1                      nrelop_usaid-laos-ev_testUpdates
4                         nrelop_usaid-laos-ev_oct9Demo
5     nrelop_usaid-laos-ev_default_NZPQovdgPI2OywmFF...
6                   nrelop_usaid-laos-ev_default_test-1
7     nrelop_usaid-laos-ev_default_jMbTtd5Qbjuw9e5BR...
8     nrelop_usaid-laos-ev_default_3JaVzbPwvJjXE8x1o...
9     nrelop_usaid-laos-ev_default_G6CznZqlNHaxJHiev...
10    nrelop_usaid-laos-ev_default_s5GK8IC6ljKwk6xZb...
11    nrelop_usaid-laos-ev_default_XTFCs7rHV0dxv2UKD...
12    nrelop_usaid-laos-ev_default_5ubmjSfP4hG09QubF...
13    nrelop_usaid-laos-ev_default_4lOYSVNYr56vXy0zS...
14              nrelop_usaid-laos-ev_testingOnTestPhone
15                nrelop_usaid-laos-ev_testingTheConfig
16    nrelop_usaid-laos-ev_default_QDnNcBapYzOCfMvCF...
17    nrelop_usaid-laos-ev_default_uVKy82hyFA88KJwUk...
18                       nrelop_usaid-laos-ev_testPhone
19    nrelop_usaid-laos-ev_default_Ioa3JqXWR42qY

Loaded all confirmed trips of length 9243


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,7,27,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,7,27,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,7,27,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,7,26,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,7,26,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09


After filtering, found 9243 participant trips 


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,7,27,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,7,27,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,7,27,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,7,26,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,7,26,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09


In [48]:
print(len(expanded_ct))

early_testing = expanded_ct[expanded_ct['start_ts'] < 1700438400]
print(len(early_testing))
print(early_testing.user_id.nunique())


expanded_ct = expanded_ct[expanded_ct['start_ts'] > 1700438400]
print(len(expanded_ct))
expanded_ct["user_id_str"] = expanded_ct.user_id.apply(lambda u: str(u))
print(len(expanded_ct.user_id_str.unique()))
expanded_ct.user_id_str.unique()

earlt_testing_all = all_trips[all_trips['start_ts'] < 1700438400]

all_trips = all_trips[all_trips['start_ts'] > 1700438400]
print(len(all_trips))
all_trips["user_id_str"] = all_trips.user_id.apply(lambda u: str(u))
print(len(all_trips.user_id_str.unique()))
# all_trips.user_id_str.unique()

1400
89
4
1311
17
6936
23


In [49]:
early_testing.user_id.unique()

array([UUID('32af51aa-eb6d-4453-ae2b-994d7d8c1fec'),
       UUID('a011eb00-9ff9-4c50-9ffb-fa44f7525bee'),
       UUID('78a8d651-dc64-4162-ba33-ee9a65dfed42'),
       UUID('d6c6cdad-4a6c-4527-9ead-c91fc3081bac')], dtype=object)

In [50]:
earlt_testing_all.user_id.unique()

array([UUID('32af51aa-eb6d-4453-ae2b-994d7d8c1fec'),
       UUID('a011eb00-9ff9-4c50-9ffb-fa44f7525bee'),
       UUID('41bd169b-9956-480c-8227-afea68db59ba'),
       UUID('5488bfaa-82a4-40d4-96fe-e6df27f0e0eb'),
       UUID('78a8d651-dc64-4162-ba33-ee9a65dfed42'),
       UUID('d6c6cdad-4a6c-4527-9ead-c91fc3081bac')], dtype=object)

## Combine program and trip tables

In [51]:
trip_program_df = expanded_ct.merge(uuid_program_df, on="user_id_str")
all_trip_df = all_trips.merge(uuid_program_df, on="user_id_str")

trip_program_df["distance_km"] = trip_program_df.distance / 1000
all_trip_df["distance_km"] = all_trip_df.distance / 1000

In [52]:
trip_program_df.sample(n=50, random_state=123)[["program", "user_id_str", "opcode", "_id", "start_fmt_time", "end_fmt_time", "distance_km", "mode_confirm"]]

,program,user_id_str,opcode,_id,start_fmt_time,end_fmt_time,distance_km,mode_confirm
569,nrelop,a1ae9652-7476-4c17-a22f-f02396e7c620,nrelop_usaid-laos-ev_default_XTFCs7rHV0dxv2UKD...,656f05299a06ac8ce99a27c7,2023-12-05T17:48:29.980606+07:00,2023-12-05T17:50:54.689000+07:00,0.289873,NaN
822,nrelop,e8dcb01c-bcb0-454d-b673-66b046319101,nrelop_usaid-laos-ev_default_uVKy82hyFA88KJwUk...,656b489385a9a8b928556757,2023-12-02T21:41:13.562164+07:00,2023-12-02T21:53:05.933395+07:00,1.331811,motorcycle
1056,nrelop,d8542bcf-64d4-41af-aa52-f17afd07335b,nrelop_usaid-laos-ev_default_Ioa3JqXWR42qYZiMD...,658ccbad10ec4115bcafa993,2023-12-28T07:17:28.603968+07:00,2023-12-28T07:46:34.997521+07:00,1.539058,motorcycle
84,nrelop,66e66174-e1be-4041-a126-c2237d81b73c,nrelop_usaid-laos-ev_default_4lOYSVNYr56vXy0zS...,6573e861a5076a5fb6421180,2023-12-09T10:37:21.794000+07:00,2023-12-09T10:46:05.908000+07:00,1.808732,motorcycle
560,nrelop,a1ae9652-7476-4c17-a22f-f02396e7c620,nrelop_usaid-laos-ev_default_XTFCs7rHV0dxv2UKD...,65685f71bf4e87728d479736,2023-11-30T16:14:44.165627+07:00,2023-11-30T16:53:53.898000+07:00,18.710056,NaN
953,nrelop,e8dcb01c-bcb0-454d-b673-66b046319101,nrelop_usaid-laos-ev_default_uVKy82hyFA88KJwUk...,65857d39cb396b4b56e436fe,2023-12-22T18:04:41.274008+07:00,2023-12-22T18:44:04.894987+07:00,9.644494,motorcycle
817,nrelop,e8dcb01c-bcb0-454d-b673-66b046319101,nrelop_usaid-laos-ev_default_uVKy82hyFA88KJwUk...,656ae624ea6bb30b37434e82,2023-12-02T10:37:17.561443+07:00,2023-12-02T10:44:52.999737+07:00,0.322619,motorcycle
1096,nrelop,d8542bcf-64d4-41af-aa52-f17afd07335b,nrelop_usaid-laos-ev_default_Ioa3JqXWR42qYZiMD...,65603e35843fb08761be8699,2023-11-24T10:55:59.631195+07:00,2023-11-24T12:56:17.057644+07:00,8.105189,motorcycle
598,nrelop,a1ae9652-7476-4c17-a22f-f02396e7c620,nrelop_usaid-laos-ev_default_XTFCs7rHV0dxv2UKD...,657a9cc8c0de892b5c82224f,2023-12-14T11:08:59.581368+07:00,2023-12-14T11:30:40.350000+07:00,1.740963,motorcycle
1249,nrelop,48cfb8eb-46eb-4c7d-80a5-9b7083b18819,nrelop_usaid-laos-ev_default_4zRn9AbfjM9KYKGl2...,65767d839dd3a34d426f3c1a,2023-12-11T07:20:52.371319+07:00,2023-12-11T08:00:55.110000+07:00,13.684881,motorcycle


In [53]:
all_trip_df.sample(n=50, random_state=123)[["program", "user_id_str", "opcode", "_id", "start_fmt_time", "end_fmt_time", "distance_km"]]

,program,user_id_str,opcode,_id,start_fmt_time,end_fmt_time,distance_km
5017,nrelop,085a73cb-74b2-48b4-8de6-98f6a4802f16,nrelop_usaid-laos-ev_default_QDnNcBapYzOCfMvCF...,6573b02e2c9dbd9468e61aab,2023-12-09T06:27:33.715256+07:00,2023-12-09T06:42:21.473000+07:00,6.083770
1999,nrelop,138d3b79-e9b3-4e7e-bf63-98ce92de740e,nrelop_usaid-laos-ev_default_3RnrZUoK2AbR4Ba4p...,658ea9b174f4718ef7404ffc,2023-12-29T16:26:07.340632+07:00,2023-12-29T17:36:34.511000+07:00,11.508358
3227,nrelop,428cbafa-7be8-4d62-b2cb-61356d0f825a,nrelop_usaid-laos-ev_default_SoaTDhidJ9IKXiDRq...,655f40e0a94f19f3184b79bf,2023-11-23T18:37:30.190352+07:00,2023-11-23T19:00:00.364231+07:00,2.093392
6023,nrelop,d8542bcf-64d4-41af-aa52-f17afd07335b,nrelop_usaid-laos-ev_default_Ioa3JqXWR42qYZiMD...,65742442d05d6813d59dfb95,2023-12-09T15:07:18.240873+07:00,2023-12-09T15:17:02.000352+07:00,0.523512
1490,nrelop,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,nrelop_usaid-laos-ev_default_3JaVzbPwvJjXE8x1o...,659bdc27f931ad418eeaf813,2024-01-08T16:54:07.903678+07:00,2024-01-08T17:57:36.890000+07:00,10.394297
6902,nrelop,21e97851-4a06-49c5-9b5c-82aca4dd9aed,nrelop_usaid-laos-ev_default_hhfF76bAEj77bAI8V...,657d22fa19a4a23ecb49565d,2023-12-16T11:01:07.873521+07:00,2023-12-16T11:04:42.708000+07:00,0.141695
3390,nrelop,8673748b-436c-4f14-8de9-dd3c3556690c,nrelop_usaid-laos-ev_default_FvYhDZQYYmrhx6eNs...,657c6c2f27e3f26191b69862,2023-12-15T20:46:07.165271+07:00,2023-12-15T21:28:31.000234+07:00,13.882664
5436,nrelop,8eea92fb-6164-48f0-94bb-37c6ce53e0c8,nrelop_usaid-laos-ev_default_UDtB2GbQkRdafA7Gn...,65b2177ecd0271f4d22dd599,2024-01-25T14:13:30.012238+07:00,2024-01-25T14:40:17.999677+07:00,12.810705
5123,nrelop,085a73cb-74b2-48b4-8de6-98f6a4802f16,nrelop_usaid-laos-ev_default_QDnNcBapYzOCfMvCF...,658ba39b9bbd2ef3a15d6317,2023-12-27T10:47:33.581000+07:00,2023-12-27T10:50:09.459000+07:00,0.152197
4604,nrelop,9ba916cb-27b0-4f76-b2c1-71c88dbf86f7,nrelop_usaid-laos-ev_default_c2Gpz3DqWrTZtvhwL...,6592491e2e46a987f87c4bff,2024-01-01T10:52:37.576098+07:00,2024-01-01T11:33:05.242960+07:00,4.171918


## Prepping dataframes for plotting

In [54]:
def all_start_end(points):
    return points.start_loc.append(points.end_loc)

def geo_start_end(start_end):
    return gpd.GeoSeries(start_end.apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")

In [55]:
print(trip_program_df.user_id.nunique())
print(all_trip_df.user_id.nunique())

all_start_end_points = all_start_end(trip_program_df)
all_modeless_points = all_start_end(all_trip_df)

all_geo_start_end_points = geo_start_end(all_start_end_points)
all_geo_modeless = geo_start_end(all_modeless_points)

earlt_testing_all_points = all_start_end(earlt_testing_all)
earlt_testing_all_points_geo = geo_start_end(earlt_testing_all_points)

17
23


/tmp/ipykernel_114/325302310.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return points.start_loc.append(points.end_loc)
/tmp/ipykernel_114/325302310.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return points.start_loc.append(points.end_loc)
/tmp/ipykernel_114/325302310.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return points.start_loc.append(points.end_loc)


In [56]:
trip_program_df.mode_confirm.unique()

array(['drove_alone', 'shared_ride', 'walk', 'motorcycle', 'taxi',
       'e-auto_rickshaw', 'e_car_drove_alone', 'not_a_trip', nan, 'bike',
       'e-bike', 'e_car_shared_ride', 'auto_rickshaw', 'air'],
      dtype=object)

In [59]:
motorcycle_trips = trip_program_df[trip_program_df.mode_confirm == 'motorcycle']; 
print(len(motorcycle_trips))
print(motorcycle_trips.user_id.nunique())

motorcycle_start_end_points = all_start_end(motorcycle_trips); 
motorcycle_geo_start_end_points = geo_start_end(motorcycle_start_end_points)

car_like_trips = trip_program_df.query('mode_confirm == "own_car" | mode_confirm == "shared_ride" | mode_confirm == "drove_alone" | mode_confirm == "e_car_drove_alone" | mode_confirm == "e_car_shared_ride" | mode_confirm == "own_cars" | mode_confirm == "taxi"'); 
print(len(car_like_trips))
print(car_like_trips.user_id.nunique())

car_like_start_end_points = all_start_end(car_like_trips)
car_like_geo_start_end_points = geo_start_end(car_like_start_end_points)

1055
10
170
10


/tmp/ipykernel_114/325302310.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return points.start_loc.append(points.end_loc)
/tmp/ipykernel_114/325302310.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return points.start_loc.append(points.end_loc)


In [35]:
trip_program_df.Trip_purpose.unique()

array(['Other', 'Pick-up/ Drop off Item', 'Home', 'To Work', 'School',
       'Meal', 'Pick-up/ Drop off Person', 'Shopping', 'At Work',
       'Personal/ Medical', 'Transit transfer', 'Religious'], dtype=object)

In [37]:
work_trips = trip_program_df[trip_program_df.Trip_purpose == 'To Work']; 
print(len(work_trips))
print(work_trips.user_id.nunique())

meal_shopping_trips = trip_program_df.query('Trip_purpose == "Shopping" | Trip_purpose == "Meal"'); 
print(len(meal_shopping_trips))
print(meal_shopping_trips.user_id.nunique())

school_trips = trip_program_df.query('Trip_purpose == "School"'); 
print(len(school_trips))
print(school_trips.user_id.nunique())

work_end_points = work_trips.end_loc; 
work_geo_end_points = geo_start_end(work_end_points)

meal_shopping_end_points = meal_shopping_trips.end_loc
meal_shopping_geo_end_points = geo_start_end(meal_shopping_end_points)

school_end_points = school_trips.end_loc
school_geo_end_points = geo_start_end(school_end_points)

681
15
64
11
124
10


### Heatmap using folium

Example from: https://stackoverflow.com/a/65756840/4040267

In [38]:
def make_basic_heatmap(all_geo_start_end):

    map = folium.Map(
        location=(17.9757, 102.6331),
        zoom_start=11,
        control_scale = True
    )
    fpl.HeatMap(all_geo_start_end.apply(lambda p: [p.y, p.x])).add_to(map)
    return map

In [39]:
#all labeled trips
make_basic_heatmap(all_geo_start_end_points)

In [40]:
make_basic_heatmap(earlt_testing_all_points_geo)

In [41]:
#all trips
make_basic_heatmap(all_geo_modeless)

In [42]:
#TEST - scaled heatmap

import branca.colormap
from collections import defaultdict
import folium
import webbrowser
from folium.plugins import HeatMap 

def make_scaled_heatmap(data):
    map_osm = folium.Map(llocation=[17.9757, 102.6331],zoom_start=11, control_scale = True)

    steps=20
    colormap = branca.colormap.linear.Spectral_09.scale(0, 1).to_step(steps)
    colormap.colors.reverse()
    gradient_map=defaultdict(dict)
    for i in range(steps):
        gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
    colormap.add_to(map_osm) #add color bar at the top of the map

    HeatMap(data.apply(lambda p: [p.y, p.x]),gradient = gradient_map).add_to(map_osm) # Add heat map to the previously created map

    return map_osm

map = make_scaled_heatmap(all_geo_modeless)
map

In [43]:
#triple heatmap for all, car, motorcycle
all_map = folium.Map(
    location=(17.9757, 102.6331),
    zoom_start=11,
    control_scale = True
)
fpl.HeatMap(all_geo_start_end_points.apply(lambda p: [p.y, p.x])).add_to(all_map)

e_bike_map = folium.Map(
    location=(17.9757, 102.6331),
    zoom_start=11,
    control_scale = True
)
fpl.HeatMap(motorcycle_geo_start_end_points.apply(lambda p: [p.y, p.x])).add_to(e_bike_map)

car_like_map = folium.Map(
    location=(17.9757, 102.6331),
    zoom_start=11,
    control_scale = True
)
fpl.HeatMap(car_like_geo_start_end_points.apply(lambda p: [p.y, p.x])).add_to(car_like_map)


fig = bre.Figure()
fig.add_subplot(1,3,1).add_child(all_map)
fig.add_subplot(1,3,2).add_child(e_bike_map)
fig.add_subplot(1,3,3).add_child(car_like_map)

In [44]:
make_basic_heatmap(work_geo_end_points)

In [45]:
make_basic_heatmap(meal_shopping_geo_end_points)

In [ ]:
make_basic_heatmap(school_geo_end_points)

### Plot all e-bike trajectories (note that this will take several hours)

In [ ]:
def get_points_for_trip(user_id, trip_id):
    ts = esta.TimeSeries.get_time_series(user_id)
    return ts.get_data_df("analysis/recreated_location",
                         time_query=esda.get_time_query_for_trip_like("analysis/confirmed_trip", trip_id))

In [ ]:
def points_map(dataframe):
    
    #create a map
    this_map = folium.Map(prefer_canvas=True, control_scale = True)

    def plotDot(point):
        '''input: series that contains a numeric named latitude and a numeric named longitude
        this function creates a CircleMarker and adds it to your this_map'''
        folium.CircleMarker(location=[point.latitude, point.longitude],
                            radius=1,
                            weight=2).add_to(this_map)

    #use df.apply(,axis=1) to "iterate" through every row in your dataframe
    dataframe.apply(plotDot, axis = 1)

    #Set the zoom to the maximum possible
    this_map.fit_bounds(this_map.get_bounds())


    return this_map

In [ ]:
def points_map_objects(point_list):
    this_map = folium.Map(prefer_canvas=True, control_scale = True)

    def plotDot(point):
        '''input: series that contains a numeric named latitude and a numeric named longitude
        this function creates a CircleMarker and adds it to your this_map'''
        folium.CircleMarker(location=[point['coordinates'][1], point['coordinates'][0]],
                            radius=1,
                            weight=2).add_to(this_map)

    #use df.apply(,axis=1) to "iterate" through every row in your dataframe
    point_list.apply(plotDot)

    #Set the zoom to the maximum possible
    this_map.fit_bounds(this_map.get_bounds())

    return this_map

In [ ]:
def get_trajectory_points(trips):
    try:
        all_trip_points = trips.apply(lambda t: get_points_for_trip(t.user_id, t._id), axis=1)
    except: #once merged with surveys
        all_trip_points = trips.apply(lambda t: get_points_for_trip(t.user_id, t._id_x), axis=1)

    all_points = functools.reduce(lambda x, y: x.append(y), all_trip_points)

    all_points_gs = gpd.GeoSeries(all_points["loc"].apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")

    return all_trip_points, all_points, all_points_gs

In [ ]:
points_map_objects(earlt_testing_all_points)

In [ ]:
#work trips
print(len(work_end_points))
points_map_objects(work_end_points)

In [ ]:
#shopping trips
print(len(meal_shopping_end_points))
points_map_objects(meal_shopping_end_points)

In [ ]:
#school trips
print(len(school_end_points))
points_map_objects(school_end_points)

In [ ]:
#using motorcycle for Laos application
e_bike_all_trip_points,  e_bike_all_points, e_bike_all_points_gs= get_trajectory_points(motorcycle_trips)

print(len(e_bike_all_points))
e_bike_all_points.head()

In [ ]:
points_map(e_bike_all_points)

In [ ]:
car_all_trip_points, car_all_points, car_all_points_gs = get_trajectory_points(car_like_trips)

print(len(car_all_points))
car_all_points.head()

In [ ]:
points_map(car_all_points)

# Gender Space!

In [60]:
surveys = pd.read_csv('PaperVizualizations/USAID-Laos-EV/usaid_laos_ev_demographics.csv')
print(len(surveys), 'surveys')

#drop any null ids
socio_data = surveys[~surveys['user_id'].isnull()]
print(len(socio_data), 'surveys after dropping null ids')

#drop duplicates
socio_data = socio_data.sort_values(by=['user_id', 'data.ts'])
socio_data.drop_duplicates(subset=['user_id'], keep='last', inplace=True)
print(len(socio_data),'surveys', socio_data['user_id'].nunique(), 'users after dropping duplicates')

#prepare survey ids for merging
socio_data['user_id_socio'] = socio_data['user_id'].astype(str)
socio_data['user_id_socio'] = socio_data['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
socio_data['user_id_socio'] = [i.replace('-','') for i in socio_data.user_id_socio]
socio_data['user_id_socio'] = socio_data['user_id_socio']
socio_data = socio_data.drop(labels='user_id', axis=1)

28 surveys
28 surveys after dropping null ids
25 surveys 25 users after dropping duplicates


In [61]:
socio_data = socio_data[socio_data['data.local_dt.timezone'] != 'America/Denver'] #throw out the test codes
print(len(socio_data))

24


In [62]:
tq = scaffolding.get_time_query(year, month)
all_trips = scaffolding.load_all_participant_trips('usaid-laos-ev', tq, False)

all_trips = all_trips[all_trips['start_ts'] > 1700438400]
all_trips["user_id_str"] = all_trips.user_id.apply(lambda u: str(u))
len(all_trips.user_id_str.unique())

all_trip_df = all_trips.merge(uuid_program_df, on="user_id_str")
all_trip_df["distance_km"] = all_trip_df.distance / 1000
all_trip_df.head()

0                   nrelop_usaid-laos-ev_testingUpdates
1                      nrelop_usaid-laos-ev_testUpdates
4                         nrelop_usaid-laos-ev_oct9Demo
5     nrelop_usaid-laos-ev_default_NZPQovdgPI2OywmFF...
6                   nrelop_usaid-laos-ev_default_test-1
7     nrelop_usaid-laos-ev_default_jMbTtd5Qbjuw9e5BR...
8     nrelop_usaid-laos-ev_default_3JaVzbPwvJjXE8x1o...
9     nrelop_usaid-laos-ev_default_G6CznZqlNHaxJHiev...
10    nrelop_usaid-laos-ev_default_s5GK8IC6ljKwk6xZb...
11    nrelop_usaid-laos-ev_default_XTFCs7rHV0dxv2UKD...
12    nrelop_usaid-laos-ev_default_5ubmjSfP4hG09QubF...
13    nrelop_usaid-laos-ev_default_4lOYSVNYr56vXy0zS...
14              nrelop_usaid-laos-ev_testingOnTestPhone
15                nrelop_usaid-laos-ev_testingTheConfig
16    nrelop_usaid-laos-ev_default_QDnNcBapYzOCfMvCF...
17    nrelop_usaid-laos-ev_default_uVKy82hyFA88KJwUk...
18                       nrelop_usaid-laos-ev_testPhone
19    nrelop_usaid-laos-ev_default_Ioa3JqXWR42qY

Loaded all confirmed trips of length 9243


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,7,27,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,7,27,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,7,27,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,7,26,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,7,26,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09


After filtering, found 9243 participant trips 


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,7,27,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,7,27,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,7,27,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,7,26,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,7,26,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts,user_id_str,program,opcode,distance_km
0,DwellSegmentationTimeFilter,1.701146e+09,2023-11-28T11:39:27.671000+07:00,"{'type': 'Point', 'coordinates': [102.6366169,...",656575fc3e348d3b5bf318e1,1.701144e+09,2023-11-28T11:06:59.564865+07:00,"{'type': 'Point', 'coordinates': [102.6255701,...",1948.106135,7298.241021,...,27,1,Asia/Vientiane,656576003e348d3b5bf31938,a74e607e-7bb3-4158-a224-9c7df6c75f7d,1.701148e+09,a74e607e-7bb3-4158-a224-9c7df6c75f7d,nrelop,nrelop_usaid-laos-ev_default_4puy9EQHKBNuJ987e...,7.298241
1,DwellSegmentationTimeFilter,1.701180e+09,2023-11-28T21:05:22.591000+07:00,"{'type': 'Point', 'coordinates': [102.6293775,...",6565f46edfacb8a4fdaa9627,1.701179e+09,2023-11-28T20:43:26.377875+07:00,"{'type': 'Point', 'coordinates': [102.6086044,...",1316.213125,6175.860731,...,22,1,Asia/Vientiane,6565f478dfacb8a4fdaa9669,a74e607e-7bb3-4158-a224-9c7df6c75f7d,1.701181e+09,a74e607e-7bb3-4158-a224-9c7df6c75f7d,nrelop,nrelop_usaid-laos-ev_default_4puy9EQHKBNuJ987e...,6.175861
2,DwellSegmentationTimeFilter,1.701164e+09,2023-11-28T16:35:14.371000+07:00,"{'type': 'Point', 'coordinates': [102.6252603,...",6565bca0618303b508374299,1.701157e+09,2023-11-28T14:40:12.467179+07:00,"{'type': 'Point', 'coordinates': [102.7409117,...",6901.903821,38793.091827,...,14,1,Asia/Vientiane,6565bca8618303b5083743b4,a74e607e-7bb3-4158-a224-9c7df6c75f7d,1.701166e+09,a74e607e-7bb3-4158-a224-9c7df6c75f7d,nrelop,nrelop_usaid-laos-ev_default_4puy9EQHKBNuJ987e...,38.793092
3,DwellSegmentationTimeFilter,1.701229e+09,2023-11-29T10:35:48.039000+07:00,"{'type': 'Point', 'coordinates': [102.6250733,...",6566b94a51c8ac1bf294a7f6,1.701224e+09,2023-11-29T09:14:20.910000+07:00,"{'type': 'Point', 'coordinates': [102.6293775,...",4887.129000,816.165187,...,48,2,Asia/Vientiane,6566b95851c8ac1bf294a8af,a74e607e-7bb3-4158-a224-9c7df6c75f7d,1.701231e+09,a74e607e-7bb3-4158-a224-9c7df6c75f7d,nrelop,nrelop_usaid-laos-ev_default_4puy9EQHKBNuJ987e...,0.816165
4,DwellSegmentationTimeFilter,1.701246e+09,2023-11-29T15:26:12.786000+07:00,"{'type': 'Point', 'coordinates': [102.6222817,...",6566ffbc28da556e24099708,1.701245e+09,2023-11-29T15:01:44.623564+07:00,"{'type': 'Point', 'coordinates': [102.6254417,...",1468.162436,7079.554286,...,12,2,Asia/Vientiane,6566ffca28da556e2409979c,a74e607e-7bb3-4158-a224-9c7df6c75f7d,1.701249e+09,a74e607e-7bb3-4158-a224-9c7df6c75f7d,nrelop,nrelop_usaid-laos-ev_default_4puy9EQHKBNuJ987e...,7.079554


In [63]:
#prepare trip ids for merging
trips = all_trip_df.copy()
print(len(trips), 'trips')
print(trips.user_id.nunique(), 'people')
trips['user_id_socio'] = trips.user_id.astype(str)
trips['user_id_socio'] = trips['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
trips.user_id_socio = [i.replace('-','') for i in trips.user_id_socio] # remove all dashes from strings

#merge the data
data = trips.merge(socio_data, on='user_id_socio')
print(len(data), 'trips after merging')
print(data.user_id_socio.nunique(), 'people after merging')

6936 trips
23 people
6324 trips after merging
22 people after merging


In [65]:
data['GENDER'] = data['What_is_your_gender']

woman = data[data.GENDER == 'woman']
print(len(woman), "woman trips")
print(woman.user_id.nunique())
man = data[data.GENDER == 'man']
print(len(man), "man trips")
print(man.user_id.nunique())

2997 woman trips
10
3327 man trips
12


In [ ]:
all_woman = all_start_end(woman)
all_geo_woman = geo_start_end(all_woman)
print(len(all_geo_woman))

all_man = all_start_end(man)
all_geo_man = geo_start_end(all_man)
print(len(all_geo_man))

In [ ]:
make_basic_heatmap(all_geo_woman)

In [ ]:
make_basic_heatmap(all_geo_man)

In [ ]:
points_map_objects(all_woman)

In [ ]:
points_map_objects(all_man)

## Gender + Mode

In [67]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dynamic_labels=dynamic_labels,
                                                                            dic_re=dic_re,
                                                                            dic_pur=dic_pur,
                                                                            include_test_users=include_test_users)

0                   nrelop_usaid-laos-ev_testingUpdates
1                      nrelop_usaid-laos-ev_testUpdates
4                         nrelop_usaid-laos-ev_oct9Demo
5     nrelop_usaid-laos-ev_default_NZPQovdgPI2OywmFF...
6                   nrelop_usaid-laos-ev_default_test-1
7     nrelop_usaid-laos-ev_default_jMbTtd5Qbjuw9e5BR...
8     nrelop_usaid-laos-ev_default_3JaVzbPwvJjXE8x1o...
9     nrelop_usaid-laos-ev_default_G6CznZqlNHaxJHiev...
10    nrelop_usaid-laos-ev_default_s5GK8IC6ljKwk6xZb...
11    nrelop_usaid-laos-ev_default_XTFCs7rHV0dxv2UKD...
12    nrelop_usaid-laos-ev_default_5ubmjSfP4hG09QubF...
13    nrelop_usaid-laos-ev_default_4lOYSVNYr56vXy0zS...
14              nrelop_usaid-laos-ev_testingOnTestPhone
15                nrelop_usaid-laos-ev_testingTheConfig
16    nrelop_usaid-laos-ev_default_QDnNcBapYzOCfMvCF...
17    nrelop_usaid-laos-ev_default_uVKy82hyFA88KJwUk...
18                       nrelop_usaid-laos-ev_testPhone
19    nrelop_usaid-laos-ev_default_Ioa3JqXWR42qY

Loaded all confirmed trips of length 9243


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,7,27,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,7,27,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,7,27,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,7,26,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,7,26,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09


After filtering, found 9243 participant trips 


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,7,27,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,7,27,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,7,27,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,7,26,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,7,26,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09


...

,purpose_confirm,mode_confirm
0,work,own_car
1,pick_drop_person,own_car
2,access_recreation,walk
3,at_work,own_car
4,exercise,own_car


Found Index(['purpose_confirm', 'mode_confirm'], dtype='object') columns of length 2
After expanding, columns went from 41 -> 43


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts,purpose_confirm,mode_confirm
0,DwellSegmentationTimeFilter,1.690422e+09,2023-07-27T08:44:00.980000+07:00,"{'type': 'Point', 'coordinates': [102.5987289,...",64c1d19fc469473186d0edf1,1.690420e+09,2023-07-27T08:14:55.523648+07:00,"{'type': 'Point', 'coordinates': [102.6310023,...",1745.456352,4967.295073,...,8,44,0,3,Asia/Vientiane,64c1d1a3c469473186d0ee4c,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690424e+09,work,own_car
1,DwellSegmentationTimeFilter,1.690457e+09,2023-07-27T18:21:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",64c25e3d4649d56e2585e88f,1.690456e+09,2023-07-27T18:00:59.484448+07:00,"{'type': 'Point', 'coordinates': [102.6582215,...",1207.232552,5632.879950,...,18,21,6,3,Asia/Vientiane,64c25e414649d56e2585e8f9,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09,pick_drop_person,own_car
2,DwellSegmentationTimeFilter,1.690458e+09,2023-07-27T18:37:54.082000+07:00,"{'type': 'Point', 'coordinates': [102.632762, ...",64c25e3d4649d56e2585e891,1.690457e+09,2023-07-27T18:24:06.717000+07:00,"{'type': 'Point', 'coordinates': [102.6322604,...",827.365000,180.494229,...,18,37,54,3,Asia/Vientiane,64c25e414649d56e2585e8fb,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690460e+09,access_recreation,walk
3,DwellSegmentationTimeFilter,1.690370e+09,2023-07-26T18:07:49.315000+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",64c10cba9efe49ad587983ab,1.690367e+09,2023-07-26T17:30:32.663000+07:00,"{'type': 'Point', 'coordinates': [102.5980728,...",2236.652000,10881.564324,...,18,7,49,2,Asia/Vientiane,64c10cc09efe49ad58798432,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09,at_work,own_car
4,DwellSegmentationTimeFilter,1.690371e+09,2023-07-26T18:29:02.825000+07:00,"{'type': 'Point', 'coordinates': [102.6475793,...",64c10cbb9efe49ad587983ad,1.690371e+09,2023-07-26T18:22:33.277876+07:00,"{'type': 'Point', 'coordinates': [102.6582093,...",389.547124,2259.165147,...,18,29,2,2,Asia/Vientiane,64c10cc09efe49ad58798434,32af51aa-eb6d-4453-ae2b-994d7d8c1fec,1.690373e+09,exercise,own_car


This is a study, not expecting any replaced modes.
_nrelop
Based on 1400 confirmed trips from 20 users
of 9243 total  trips from 27 users (15.15%)


0                   nrelop_usaid-laos-ev_testingUpdates
1                      nrelop_usaid-laos-ev_testUpdates
4                         nrelop_usaid-laos-ev_oct9Demo
5     nrelop_usaid-laos-ev_default_NZPQovdgPI2OywmFF...
6                   nrelop_usaid-laos-ev_default_test-1
7     nrelop_usaid-laos-ev_default_jMbTtd5Qbjuw9e5BR...
8     nrelop_usaid-laos-ev_default_3JaVzbPwvJjXE8x1o...
9     nrelop_usaid-laos-ev_default_G6CznZqlNHaxJHiev...
10    nrelop_usaid-laos-ev_default_s5GK8IC6ljKwk6xZb...
11    nrelop_usaid-laos-ev_default_XTFCs7rHV0dxv2UKD...
12    nrelop_usaid-laos-ev_default_5ubmjSfP4hG09QubF...
13    nrelop_usaid-laos-ev_default_4lOYSVNYr56vXy0zS...
14              nrelop_usaid-laos-ev_testingOnTestPhone
15                nrelop_usaid-laos-ev_testingTheConfig
16    nrelop_usaid-laos-ev_default_QDnNcBapYzOCfMvCF...
17    nrelop_usaid-laos-ev_default_uVKy82hyFA88KJwUk...
18                       nrelop_usaid-laos-ev_testPhone
19    nrelop_usaid-laos-ev_default_Ioa3JqXWR42qY

In [68]:
#prepare trip ids for merging
trips = expanded_ct.copy()
print(len(trips), 'trips')
print(trips.user_id.nunique(), 'people')
trips['user_id_socio'] = trips.user_id.astype(str)
trips['user_id_socio'] = trips['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
trips.user_id_socio = [i.replace('-','') for i in trips.user_id_socio] # remove all dashes from strings

#merge the data
data = trips.merge(socio_data, on='user_id_socio')
print(len(data), 'trips after merging')
print(data.user_id_socio.nunique(), 'people after merging')

1400 trips
20 people
1312 trips after merging
18 people after merging


In [69]:
data.columns

Index(['source', 'end_ts', 'end_fmt_time', 'end_loc', 'raw_trip', 'start_ts',
       'start_fmt_time', 'start_loc', 'duration', 'distance', 'start_place',
       'end_place', 'cleaned_trip', 'inferred_labels', 'inferred_trip',
       'expectation', 'confidence_threshold', 'expected_trip',
       'inferred_section_summary', 'cleaned_section_summary', 'user_input',
       'additions', 'start_local_dt_year', 'start_local_dt_month',
       'start_local_dt_day', 'start_local_dt_hour', 'start_local_dt_minute',
       'start_local_dt_second', 'start_local_dt_weekday',
       'start_local_dt_timezone', 'end_local_dt_year', 'end_local_dt_month',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_second', 'end_local_dt_weekday', 'end_local_dt_timezone',
       '_id_x', 'user_id', 'metadata_write_ts', 'purpose_confirm',
       'mode_confirm', 'distance_miles', 'Mode_confirm', 'Trip_purpose',
       'user_id_socio', '_id_y', 'data.ts', 'At_your_primary_job_

In [70]:
motorcycle = data[data["mode_confirm"] == "motorcycle"]
print(len(motorcycle), "total motorcycle trips")

woman_motorcycle = motorcycle[motorcycle["What_is_your_gender"] == "woman"]
print(len(woman_motorcycle), "by women")
print(woman_motorcycle.user_id.nunique())

man_motorcycle = motorcycle[motorcycle["What_is_your_gender"] == "man"]
print(len(man_motorcycle), "by men")
print(man_motorcycle.user_id.nunique())

1055 total motorcycle trips
659 by women
5
396 by men
5


In [71]:
all_woman_cycle = all_start_end(woman_motorcycle)
all_geo_woman_cycle = geo_start_end(all_woman_cycle)
print(len(all_geo_woman_cycle))

all_man_cycle = all_start_end(man_motorcycle)
all_geo_man_cycle = geo_start_end(all_man_cycle)
print(len(all_geo_man_cycle))

1318
792


/tmp/ipykernel_114/325302310.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return points.start_loc.append(points.end_loc)
/tmp/ipykernel_114/325302310.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return points.start_loc.append(points.end_loc)


In [72]:
woman_motorcycle.columns

Index(['source', 'end_ts', 'end_fmt_time', 'end_loc', 'raw_trip', 'start_ts',
       'start_fmt_time', 'start_loc', 'duration', 'distance', 'start_place',
       'end_place', 'cleaned_trip', 'inferred_labels', 'inferred_trip',
       'expectation', 'confidence_threshold', 'expected_trip',
       'inferred_section_summary', 'cleaned_section_summary', 'user_input',
       'additions', 'start_local_dt_year', 'start_local_dt_month',
       'start_local_dt_day', 'start_local_dt_hour', 'start_local_dt_minute',
       'start_local_dt_second', 'start_local_dt_weekday',
       'start_local_dt_timezone', 'end_local_dt_year', 'end_local_dt_month',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_second', 'end_local_dt_weekday', 'end_local_dt_timezone',
       '_id_x', 'user_id', 'metadata_write_ts', 'purpose_confirm',
       'mode_confirm', 'distance_miles', 'Mode_confirm', 'Trip_purpose',
       'user_id_socio', '_id_y', 'data.ts', 'At_your_primary_job_

In [73]:
all_woman_cycle_trip_points, all_woman_cycle_points,  all_woman_cycle_points_gs = get_trajectory_points(woman_motorcycle)

print(len(all_woman_cycle_points))
all_woman_cycle_points.head()

NameError: name 'get_trajectory_points' is not defined

In [74]:
points_map(all_woman_cycle_points)

NameError: name 'points_map' is not defined

In [ ]:
all_man_cycle_trip_points, all_man_cycle_points, all_man_cycle_points_gs = get_trajectory_points(man_motorcycle)

print(len(all_man_cycle_points))
all_man_cycle_points.head()

In [ ]:
points_map(all_man_cycle_points)

# Gender + Purpose

In [75]:
data.Trip_purpose.unique()

array(['Other', 'Pick-up/ Drop off Item', 'Home', 'To Work', 'School',
       'Meal', 'Pick-up/ Drop off Person', 'Shopping', 'At Work',
       'Personal/ Medical', 'Transit transfer', 'Religious'], dtype=object)

In [76]:
work_trips = data[data['Trip_purpose'] == "To Work"]
print(len(work_trips), " total work trips")

woman_work = work_trips[work_trips["What_is_your_gender"] == "woman"]
print(len(woman_work), " by women")
print(woman_work.user_id.nunique())

man_work = work_trips[work_trips["What_is_your_gender"] == "man"]
print(len(man_work), "by men")
print(man_work.user_id.nunique())

681  total work trips
427  by women
7
254 by men
9


In [ ]:
women_work_end_points = woman_work.end_loc; 
women_work_geo_end_points = geo_start_end(women_work_end_points)

men_work_end_points = man_work.end_loc; 
men_work_geo_end_points = geo_start_end(men_work_end_points)

In [ ]:
print(len(women_work_geo_end_points))
make_basic_heatmap(women_work_geo_end_points)

In [ ]:
print(len(men_work_geo_end_points))
make_basic_heatmap(men_work_geo_end_points)

## Experimenting with whether we can associate other fields with the start_end_loc points

In [ ]:
start_loc_df = trip_program_df[["program", "mode_confirm", "start_loc"]].copy()
start_loc_df["type"] = ["start"] * len(start_loc_df)
start_loc_df.rename(columns = {"start_loc": "location"}, inplace=True)
start_loc_df.head()

In [ ]:
end_loc_df = trip_program_df[["program", "mode_confirm", "end_loc"]].copy()
end_loc_df["type"] = ["end"] * len(end_loc_df)
end_loc_df.rename(columns = {"end_loc": "location"}, inplace=True)
end_loc_df.head()

In [ ]:
loc_program_df = gpd.GeoDataFrame(start_loc_df.append(end_loc_df))
loc_program_geodf = loc_program_df.set_geometry(loc_program_df['location'].apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")
loc_program_geodf.head()

In [ ]:
program_endpoints_within = loc_program_geodf[loc_program_geodf.within(denver_boundary.geometry.iloc[0])]

In [ ]:
program_endpoints_within.head()

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(12,6))
denver_boundary.boundary.plot(ax=ax_arr[0])
program_endpoints_within.plot(column="program", cmap="tab10", ax=ax_arr[0])
denver_boundary.boundary.plot(ax=ax_arr[1])
program_endpoints_within.query('mode_confirm == "pilot_ebike"').plot(column="program", cmap="tab10", ax=ax_arr[1])
denver_boundary.boundary.plot(ax=ax_arr[2])
program_endpoints_within.query('mode_confirm == "drove_alone" | mode_confirm == "shared_ride" | mode_confirm == "taxi"').plot(column="program", cmap="tab10", ax=ax_arr[2])

In [ ]:
other_programs_endpoints_within_query = program_endpoints_within.query('program != "sc" & program != "prepilot" & program != "stage"')

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(20,10))
denver_boundary.boundary.plot(ax=ax_arr[0])
other_programs_endpoints_within_query.plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[0], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[0].set_title("all")
denver_boundary.boundary.plot(ax=ax_arr[1])
other_programs_endpoints_within_query.query('mode_confirm == "pilot_ebike"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[1], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[1].set_title("e-bike")
denver_boundary.boundary.plot(ax=ax_arr[2])
other_programs_endpoints_within_query.query('mode_confirm == "drove_alone"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[2], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[2].set_title("drove_alone")
denver_boundary.boundary.plot(ax=ax_arr[3])
other_programs_endpoints_within_query.query('mode_confirm == "shared_ride"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[3], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[3].set_title("shared_ride")
denver_boundary.boundary.plot(ax=ax_arr[4])
other_programs_endpoints_within_query.query('mode_confirm == "taxi"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[4], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[4].set_title("taxi")
# ax_arr[0].legend(["denver"] + other_programs_endpoints_within_query.program.unique().tolist(), loc="lower left", bbox_to_anchor=(0.5, 0.5))

In [ ]:
program_endpoints_within.to_file("ceo_results/program_endpoints_within_denver")

In [ ]:
program_endpoints_within.to_csv("ceo_results/program_endpoints_within_denver.csv")

In [ ]:
sc_pixel_stats_to_export = sc_pixel_stats.copy()

In [ ]:
sc_pixel_stats_to_export = sc_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
sc_pixel_stats_to_export.to_file("ceo_results/sc_pixel_stats")

In [ ]:
prepilot_pixel_stats_to_export = prepilot_pixel_stats.copy()

In [ ]:
prepilot_pixel_stats_to_export = prepilot_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
prepilot_pixel_stats_to_export.to_file("ceo_results/prepilot_pixel_stats")

In [ ]:
sc_pixel_stats.to_csv("ceo_results/sc_pixel_stats.csv")

In [ ]:
prepilot_pixel_stats.to_csv("ceo_results/prepilot_pixel_stats.csv")

In [ ]:
other_programs_pixel_stats_to_export = other_programs_pixel_stats.copy()

In [ ]:
other_programs_pixel_stats_to_export = other_programs_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
other_programs_pixel_stats_to_export.to_file("ceo_results/other_programs_pixel_stats")

In [ ]:
other_programs_pixel_stats.to_csv("ceo_results/other_programs_pixel_stats.csv")